In [1]:
#HYPER PARAMETER OPTIMIZATION ON CNN MODELS 

In [6]:
#pip install scikit-learn

In [14]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split

In [10]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 22s 1us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 7s 2us/step


In [12]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [20]:
def build_model(optimizer='adam', dropout_rate=0.0):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [32]:
def train_with_params(optimizer_type, dropout_rate, epochs, batch_size):
    # Create a new optimizer instance for each training
    if optimizer_type == 'adam':
        optimizer = Adam(learning_rate=0.001)
    elif optimizer_type == 'sgd':
        optimizer = SGD(learning_rate=0.01)
    model = build_model(optimizer, dropout_rate)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Optimizer: {optimizer_type}, Dropout: {dropout_rate}, Epochs: {epochs}, Batch Size: {batch_size}, Validation Accuracy: {val_acc}")
    return val_acc

In [34]:
optimizers = ['adam', 'sgd']
dropout_rates = [0.0, 0.2, 0.4]
epochs_list = [10, 20]
batch_sizes = [32, 64]

In [36]:
best_acc = 0
best_params = None
for optimizer in optimizers:
    for dropout_rate in dropout_rates:
        for epochs in epochs_list:
            for batch_size in batch_sizes:
                val_acc = train_with_params(optimizer, dropout_rate, epochs, batch_size)
                if val_acc > best_acc:
                    best_acc = val_acc
                    best_params = (optimizer, dropout_rate, epochs, batch_size)

Optimizer: adam, Dropout: 0.0, Epochs: 10, Batch Size: 32, Validation Accuracy: 0.9167500138282776
Optimizer: adam, Dropout: 0.0, Epochs: 10, Batch Size: 64, Validation Accuracy: 0.909500002861023
Optimizer: adam, Dropout: 0.0, Epochs: 20, Batch Size: 32, Validation Accuracy: 0.906333327293396
Optimizer: adam, Dropout: 0.2, Epochs: 10, Batch Size: 32, Validation Accuracy: 0.9080833196640015
Optimizer: adam, Dropout: 0.2, Epochs: 10, Batch Size: 64, Validation Accuracy: 0.9114999771118164
Optimizer: adam, Dropout: 0.2, Epochs: 20, Batch Size: 32, Validation Accuracy: 0.9104166626930237
Optimizer: adam, Dropout: 0.2, Epochs: 20, Batch Size: 64, Validation Accuracy: 0.9160833358764648
Optimizer: adam, Dropout: 0.4, Epochs: 10, Batch Size: 32, Validation Accuracy: 0.9117500185966492
Optimizer: adam, Dropout: 0.4, Epochs: 10, Batch Size: 64, Validation Accuracy: 0.906333327293396
Optimizer: adam, Dropout: 0.4, Epochs: 20, Batch Size: 32, Validation Accuracy: 0.9154999852180481
Optimizer: ad

In [38]:
optimizer, dropout_rate, epochs, batch_size = best_params
final_model = build_model(optimizer, dropout_rate)
final_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.7487 - loss: 0.6975 - val_accuracy: 0.8704 - val_loss: 0.3627
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - accuracy: 0.8786 - loss: 0.3365 - val_accuracy: 0.8806 - val_loss: 0.3169
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.8980 - loss: 0.2782 - val_accuracy: 0.8888 - val_loss: 0.3048
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9114 - loss: 0.2391 - val_accuracy: 0.9006 - val_loss: 0.2684
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.9186 - loss: 0.2162 - val_accuracy: 0.9002 - val_loss: 0.2590
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - accuracy: 0.9287 - loss: 0.1868 - val_accuracy: 0.9080 - val_loss: 0.2543
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - accuracy: 0.9363 - loss: 0.1690 - val_accuracy: 0.9071 - val_loss: 0.2696
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.9445 -

In [40]:
test_loss, test_acc = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9092 - loss: 0.3156
Test Accuracy: 0.9089000225067139
